In [43]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from langchain_groq import ChatGroq
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import os
import fitz
import pdfplumber
import json
import ollama

In [44]:
documents_folder = 'documents'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
document_files = ['COSCO.pdf']
document_path = os.path.join(documents_folder, document_files[0])


sample_prompt = """
Your task is to extract the below fields in the same format from the text mentioned after format:
{{
    "fileName": "",
    "orgName": "",
    "orderId": "",
    "ocrBucketName": "",
    "docType": "",
    "bookingDetails": {{
        "carrierBookingNum": "",
        "billOfLadingNum": "",
        "referenceNumber": "",
        "carrierName": "",
        "originServiceMode": "",
        "destinationServiceMode": "",
        "placeOfOrigin": "",
        "placeOfDelivery": "",
        "portOfLoad": "",
        "portOfDischarge": "",
        "temperature": {{
            "unit": "",
            "value": ""
        }},
        "humidity": "",
        "gensetRequired": "",
        "travelType": "",
        "containerDetails": {{
            "code": "",
            "quantity": ""
        }},
        "weight": {{
            "unit": "",
            "value": ""
        }},
        "commodityType": "",
        "hazardous": {{
            "hazCode": "",
            "imoClassType": "",
            "packageCount": "",
            "packageType": "",
            "packageGroup": ""
        }},
        "voyageInfo": {{
            "vesselName": "",
            "voyageNumber": "",
            "imoNumber": "",
            "departureEstimated": "",
            "arrivalEstimated": ""
        }}
    }},
    "shipmentDetails": {{
        "shipmentDate": "",
        "containerPickUpDate": "",
        "railCutOffDate": "",
        "portCutOffDate": "",
        "vgmCutoffDate": "",
        "portOpenDate": "",
        "siCutOffDate": "",
        "shipOnBoardDate": "",
        "vent": ""
    }},
    "containers": [{{
        "containerId": "",
        "type": ""
    }}, {{
        "containerId": "",
        "type": ""
    }}]
}}
    
Text for information extraction:

{text}

Make sure to format the result in the specified format.

Just give the JSON result. No extra info
"""


In [45]:
def extract_text_from_pdf_direct(state):
    pdf_path=state.get("file_path")
    all_text = []

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Loop through each page
        for page in pdf.pages:
            # Extract text from the current page
            page_text = page.extract_text()
            if page_text:
                all_text.append(page_text)

    # Join all text from all pages
    complete_text = "\n".join(all_text)
    return {'response':complete_text}

In [46]:
prompt = PromptTemplate(template=sample_prompt)
llm = ChatGroq(model="llama3-70b-8192",groq_api_key="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")
class GraphState(TypedDict):
    file_path: Optional[str]
    response: Optional[str]
wf = StateGraph(GraphState)
def process_text(state):
    current_response = state.get('response')
    chain = prompt | llm 
    result = chain.invoke({"text": current_response})
    return {'response': result}

wf.add_node("node_1", extract_text_from_pdf_direct)
wf.add_node("node_2", process_text)
wf.set_entry_point("node_1")
wf.add_edge("node_1", "node_2")
wf.set_finish_point("node_2")

app = wf.compile()



documents_folder = 'documents'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
document_files = ['COSCO.pdf']
document_path = os.path.join(documents_folder, document_files[0])

output = app.invoke({'file_path': document_path})
print(output['response'].content)

{
    "fileName": "",
    "orgName": "COSCO SHIPPING Lines (North America) Inc.",
    "orderId": "6343267510",
    "ocrBucketName": "",
    "docType": "Booking Acknowledgement",
    "bookingDetails": {
        "carrierBookingNum": "6343267510",
        "billOfLadingNum": "",
        "referenceNumber": "LAN22039",
        "carrierName": "COSCO SHIPPING Lines (North America) Inc.",
        "originServiceMode": "CY",
        "destinationServiceMode": "CY",
        "placeOfOrigin": "Long Beach,Los Angeles, California, United States",
        "placeOfDelivery": "Hong Kong,Hong Kong",
        "portOfLoad": "Long Beach / Long Beach Container Terminal , LLC",
        "portOfDischarge": "Hong Kong / Cosco-Hit Terminals (Hong Kong) Ltd",
        "temperature": {
            "unit": "F",
            "value": "68"
        },
        "humidity": "",
        "gensetRequired": "Clip-on",
        "travelType": "FCL",
        "containerDetails": {
            "code": "40' Hi-Cube Refrigerated Container

In [47]:

def automate_text(state):
    with open(state.get("file_path")) as temp:
        text=temp.read()
    
    return {"response":text}

prompt = PromptTemplate(template=sample_prompt)
llm = ChatGroq(model="llama3-70b-8192",groq_api_key="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")
class GraphState(TypedDict):
    file_path: Optional[str]
    response: Optional[str]
wf = StateGraph(GraphState)
def process_text(state):
    current_response = state.get('response')
    chain = prompt | llm 
    result = chain.invoke({"text": current_response})
    return {'response': result}

wf.add_node("node_1", automate_text)
wf.add_node("node_2", process_text)
wf.set_entry_point("node_1")
wf.add_edge("node_1", "node_2")
wf.set_finish_point("node_2")

app = wf.compile()



documents_folder = 'OUT'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
document_files = ['COSCO.txt']
document_path = os.path.join(documents_folder, document_files[0])

output = app.invoke({'file_path': document_path})
res=output['response'].content
res=json.loads(res)
res['fileName']=document_files[0]
print(output['response'].content)

{
    "fileName": "FWK2-2022082620431432-54",
    "orgName": "COSCO SHIPPING Lines (North America) Inc.",
    "orderId": "6343267510",
    "ocrBucketName": "",
    "docType": "Booking Acknowledgement",
    "bookingDetails": {
        "carrierBookingNum": "6343267510",
        "billOfLadingNum": "",
        "referenceNumber": "LAN22039",
        "carrierName": "COSCO SHIPPING Lines (North America) Inc.",
        "originServiceMode": "FCL",
        "destinationServiceMode": "FCL",
        "placeOfOrigin": "Long Beach,Los Angeles, California, United States",
        "placeOfDelivery": "Hong Kong,Hong Kong",
        "portOfLoad": "Long Beach / Long Beach Container Terminal , LLC",
        "portOfDischarge": "Hong Kong / Cosco-Hit Terminals (Hong Kong) Ltd",
        "temperature": {
            "unit": "F",
            "value": "68"
        },
        "humidity": "",
        "gensetRequired": "Clip-on",
        "travelType": "",
        "containerDetails": {
            "code": "40' Hi-Cube

In [48]:
import objc
from Cocoa import NSURL
from Quartz import CIImage
from Vision import VNImageRequestHandler, VNRecognizeTextRequest, VNRecognizedTextObservation

def recognize_text_in_image(state):
    image_path=state.get("file_path")
    # Convert image file path to NSURL
    image_url = NSURL.fileURLWithPath_(image_path)
    
    # Create a CIImage from the image file
    ci_image = CIImage.imageWithContentsOfURL_(image_url)
    
    if ci_image is None:
        print(f"Failed to load image from {image_path}")
        return None
    
    # Create a text recognition request
    recognized_texts = []

    def handle_text_recognition_request(request, error):
        if error is not None:
            print(f"Error: {error.localizedDescription()}")
            return
        for observation in request.results():
            if isinstance(observation, VNRecognizedTextObservation):
                top_candidate = observation.topCandidates_(1)[0]
                recognized_texts.append(top_candidate.string())
    
    request = VNRecognizeTextRequest.alloc().initWithCompletionHandler_(handle_text_recognition_request)
    
    # Create an image request handler
    handler = VNImageRequestHandler.alloc().initWithCIImage_options_(ci_image, None)
    
    # Perform the request
    success, error = handler.performRequests_error_([request], None)
    
    if not success:
        print(f"Failed to perform text recognition request: {error.localizedDescription()}")
        return None

    # Join all recognized text parts into a single string
    k= "\n".join(recognized_texts)
    return {"response":k}



In [52]:
prompt = PromptTemplate(template=sample_prompt)
llm = ChatGroq(model="llama3-70b-8192",groq_api_key="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")
class GraphState(TypedDict):
    file_path: Optional[str]
    response: Optional[str]
wf = StateGraph(GraphState)
def process_text(state):
    current_response = state.get('response')
    chain = prompt | llm 
    result = chain.invoke({"text": current_response})
    return {'response': result}

wf.add_node("node_1", recognize_text_in_image)
wf.add_node("node_2", process_text)
wf.set_entry_point("node_1")
wf.add_edge("node_1", "node_2")
wf.set_finish_point("node_2")

app = wf.compile()



documents_folder = 'documents'  # Change this to your documents folder path
document_files = [f for f in os.listdir(documents_folder) if os.path.isfile(os.path.join(documents_folder, f))]
document_files = ['COSCO.pdf']
document_path = os.path.join(documents_folder, document_files[0])

output = app.invoke({'file_path': 'Screenshot 2024-07-02 at 11.49.24 PM.png'})
print(output['response'].content)

{
    "fileName": "",
    "orgName": "COSCO SHIPPING LINES CO., LTD",
    "orderId": "634326/510",
    "ocrBucketName": "",
    "docType": "Booking Acknowledgement",
    "bookingDetails": {
        "carrierBookingNum": "6343267510",
        "billOfLadingNum": "",
        "referenceNumber": "",
        "carrierName": "COSCO SHIPPING Lines (North America Inc.)",
        "originServiceMode": "",
        "destinationServiceMode": "",
        "placeOfOrigin": "",
        "placeOfDelivery": "",
        "portOfLoad": "Hong Kong Hong Ko HI Terminals (Hong Kong) Li",
        "portOfDischarge": "Long Beach Container Terminal",
        "temperature": {
            "unit": "F",
            "value": "68"
        },
        "humidity": "0 pct",
        "gensetRequired": "",
        "travelType": "",
        "containerDetails": {
            "code": "6 × 10 Hi Cube Refrigerated Container",
            "quantity": "6"
        },
        "weight": {
            "unit": "KG",
            "value": "9071.

In [50]:
"""brew services start mongodb/brew/mongodb-community"""
"use in terminal"

'use in terminal'

In [51]:
import json
from pymongo import MongoClient

connection_string = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.2.10"


# Connect to MongoDB
client = MongoClient(connection_string)

db = client['Document_Reader']  # Replace 'your_database' with the name of your database
collection = db['doc_to_json_1']  # Replace 'your_collection' with the name of your collection
if isinstance(res, list):
    collection.insert_many(res)
else:
    collection.insert_one(res)

